In [1]:
from pyspark.sql import SparkSession

from sparksampling.config import SPARK_CONF

conf = SPARK_CONF
spark = SparkSession.builder.config(conf=conf).getOrCreate()


/root/sparksampling/venv/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df = spark.read.csv("ten_million_top1k.csv", header=True).toPandas()

In [6]:
y = df[['y']]
y

,y
0,0
1,0
2,0
3,1
4,0
...,...
995,0
996,1
997,0
998,0


In [12]:
X = df.drop(columns=['# id','y'], axis=1)
X.values

array([['6', '5', '4', ..., '-3.6292718', '2.1533357000000004',
        '0.21443366'],
       ['6', '2', '0', ..., '0.61825443', '-0.20594382',
        '1.0061347999999999'],
       ['4', '2', '0', ..., '-0.35092328', '0.35849345', '0.91475412'],
       ...,
       ['6', '2', '3', ..., '0.17274349', '0.083568695', '0.85332849'],
       ['6', '5', '3', ..., '1.7405881', '-0.81428724', '-0.93481414'],
       ['0', '6', '2', ..., '0.42450829999999995',
        '-0.0017179298999999999', '-3.8725715']], dtype=object)

In [19]:
from imblearn.under_sampling import EditedNearestNeighbours
enn = EditedNearestNeighbours()
x_fit, y_fit = enn.fit_resample(X.values, y.values)

In [24]:
import pandas as pd
pd.DataFrame(x_fit, columns = X.columns)

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_90,X_91,X_92,X_93,X_94,X_95,X_96,X_97,X_98,X_99
0,6.0,5.0,4.0,5.0,2.0,2.0,3.0,3.0,6.0,5.0,...,-1.713142,2.951669,-1.608283,-2.687212,-0.809006,0.679043,2.420450,-3.629272,2.153336,0.214434
1,6.0,2.0,0.0,4.0,5.0,2.0,6.0,3.0,6.0,0.0,...,1.223221,0.446508,0.651251,-0.727004,2.328492,0.697474,0.656398,0.618254,-0.205944,1.006135
2,4.0,2.0,0.0,4.0,6.0,3.0,3.0,3.0,6.0,0.0,...,1.015165,1.172190,0.741630,-1.366551,-0.656233,1.058246,1.232354,-0.350923,0.358493,0.914754
3,4.0,6.0,3.0,5.0,2.0,2.0,6.0,1.0,1.0,0.0,...,-2.333362,1.060832,-1.199539,-0.961484,2.144502,0.723086,0.866016,-1.320033,0.781048,0.159866
4,0.0,5.0,2.0,4.0,6.0,2.0,3.0,5.0,2.0,1.0,...,-2.972420,0.897007,-0.759644,-1.092099,-0.534969,-0.384158,0.985030,-0.099703,0.197429,-0.078188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,3.0,0.0,5.0,4.0,6.0,5.0,4.0,2.0,1.0,1.0,...,-0.696159,-0.839109,-1.309599,1.005017,1.649397,-1.136458,-0.906425,0.153697,-0.212211,-2.378260
944,0.0,6.0,5.0,4.0,6.0,5.0,6.0,6.0,0.0,4.0,...,-0.218904,-1.410298,-0.492720,1.699682,-0.261111,-0.654157,-1.532982,0.219928,-0.339177,0.989774
945,1.0,0.0,0.0,4.0,6.0,6.0,6.0,3.0,1.0,1.0,...,0.204937,-1.631751,1.633170,2.091174,0.541196,-0.266991,-1.886537,-0.199328,-0.185737,0.185880
946,1.0,4.0,5.0,3.0,5.0,6.0,2.0,1.0,3.0,2.0,...,-0.881256,0.427251,1.080183,-0.409996,-0.011785,0.343887,0.369398,-0.448761,0.276815,0.651438


In [26]:
pd.DataFrame(y_fit, columns = y.columns)

,y
0,0
1,0
2,0
3,0
4,0
...,...
943,1
944,1
945,1
946,1


In [30]:
resultdf = pd.concat([pd.DataFrame(x_fit, columns = X.columns), pd.DataFrame(y_fit, columns = y.columns)], axis=1)

In [31]:
spark.createDataFrame(resultdf)

DataFrame[X_0: double, X_1: double, X_2: double, X_3: double, X_4: double, X_5: double, X_6: double, X_7: double, X_8: double, X_9: double, X_10: double, X_11: double, X_12: double, X_13: double, X_14: double, X_15: double, X_16: double, X_17: double, X_18: double, X_19: double, X_20: double, X_21: double, X_22: double, X_23: double, X_24: double, X_25: double, X_26: double, X_27: double, X_28: double, X_29: double, X_30: double, X_31: double, X_32: double, X_33: double, X_34: double, X_35: double, X_36: double, X_37: double, X_38: double, X_39: double, X_40: double, X_41: double, X_42: double, X_43: double, X_44: double, X_45: double, X_46: double, X_47: double, X_48: double, X_49: double, X_50: double, X_51: double, X_52: double, X_53: double, X_54: double, X_55: double, X_56: double, X_57: double, X_58: double, X_59: double, X_60: double, X_61: double, X_62: double, X_63: double, X_64: double, X_65: double, X_66: double, X_67: double, X_68: double, X_69: double, X_70: double, X_71: 